# Basic functionality

In this notebook we demonstrate the basic usage of the MICROtool package. The main usage of MICROtool is to optimize a diffusion imaging acquisition scheme based on a microstructural model. In order to do so the steps to follow are
1. Setup a tissue model
2. Setup an acquisition scheme (it is recommended to choose a scheme to already is a sensible choice for your problem)
3. Optimize the scheme using cramer rao lower bound based optimization
4. (optional) Verify the resulting scheme using monte carlo simulations.


## Setting up the dmipy model

We demonstrate this using a simple model from dmipy

In [1]:
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models.gaussian_models import G1Ball

from microtool.dmipy import DmipyTissueModel
from microtool.tissue_model import RelaxedMultiTissueModel

# setting up a simple model
simple_ball = G1Ball(lambda_iso=1.7e-9)

# This step is unnecessary
dmipy_model = MultiCompartmentModel([simple_ball])

microtool_model = DmipyTissueModel(dmipy_model)

## Setting up the acquisition scheme

In [2]:
from microtool.dmipy import convert_dmipy_scheme2diffusion_scheme

from microtool.utils.saved_schemes import alexander2008_optimized_directions

dmipy_scheme = alexander2008_optimized_directions([2, 8, 16])
initial_scheme = convert_dmipy_scheme2diffusion_scheme(dmipy_scheme)
initial_scheme.fix_b0_measurements()
print(initial_scheme)

Acquisition scheme with 44 measurements and 6 scalar parameters:
  DiffusionBValue [s/mm²] in (0.0, 20000.0)    DiffusionGradientAnglePhi [rad] (fixed)    DiffusionGradientAngleTheta [rad] (fixed)    DiffusionPulseWidth [ms] in (1.0, 100.0)    DiffusionPulseInterval [ms] in (1.0, 1000.0)    EchoTime [ms] in (0.1, 1000.0)
-------------------------------------------  -----------------------------------------  -------------------------------------------  ------------------------------------------  ----------------------------------------------  --------------------------------
                                          0                                 -1.96142                                       1.98074                                            7                                              12                           36.2367
                                          0                                  2.96645                                       0.952638                              

Now that we have the acquisition scheme and tissue model defined we test the generation of signal.

In [3]:
microtool_model(initial_scheme)

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.26539597e-01, 1.26539597e-01,
       2.27448870e-03, 2.27448870e-03, 2.27448870e-03, 2.27448870e-03,
       2.27448870e-03, 2.27448870e-03, 2.27448870e-03, 2.27448870e-03,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13])

## Optimizing the scheme with respect to the diffusivity.

In [4]:
from microtool.optimize import optimize_scheme

optimal_scheme, _ = optimize_scheme(initial_scheme, microtool_model, noise_variance=0.02, method=None)

C:\Users\Frank\miniconda3\envs\devmicro\lib\site-packages\scipy\optimize\_constraints.py:446: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


## Fitting the model

In [5]:
# Generating basic signal on the new scheme
signal = microtool_model(optimal_scheme)

fitted_model = microtool_model.fit(optimal_scheme, signal)
fitted_model.fitted_parameters

Using parallel processing with 12 workers.
Setup brute2fine optimizer in 0.005052328109741211 seconds
Fitting of 1 voxels complete in 3.864222764968872 seconds.
Average of 3.864222764968872 seconds per voxel.


{'G1Ball_1_lambda_iso': array([1.69996878e-09])}

## Starting an experiment

This is an optional step to initiate some directories

In [1]:
from microtool.monte_carlo.IO import make_expirement_directories
from pathlib import Path

# make an expirement directory in the current execution directory
_, _, simdir, _ = make_expirement_directories(Path.cwd(), "basic_output")

C:\Users\Frank\OneDrive - Universiteit Utrecht\Documents\Major Research\Software\microtool\microtool\monte_carlo\IO.py:42: UserWarning: There is already a folder with the name of the experiment. No new folders have been created.
  warnings.warn("There is already a folder with the name of the experiment. No new folders have been created.")


## Validation of scheme optimization through Monte Carlo simulations

What we want to do now is verify that our scheme increase the accuracy with which we can fit the parameters of our tissue model. To do so we run monte carlo simulations both with the scheme we started out with and the scheme resulting from the optimization.

In [ ]:
from microtool.monte_carlo.simulation import MonteCarloSimulation
from microtool.monte_carlo.parameter_distributions import plot_parameter_distributions
from scipy import stats

# loc is mean and scale is standard deviation
noise_distribution = stats.norm(loc=0, scale=0.02)

simulation = MonteCarloSimulation(initial_scheme, microtool_model, noise_distribution, n_sim=100)
simulation.run()

# Its wise to save your simulations, always check before running a simulation that the path is valid! (write a string to it for example)
simulation.save(simdir / "optimal.pkl")

# Doing the same for a simulation with the initial scheme
simulation.set_scheme(initial_scheme)
simulation.run()
simulation.save(simdir / "non_optimal.pkl")

In [ ]:
from microtool.utils.IO import get_pickle

result = get_pickle(simdir / "non_optimal.pkl")
fig = plot_parameter_distributions(result, microtool_model)

In [ ]:
result = get_pickle(simdir / "optimal.pkl")
fig = plot_parameter_distributions(result, microtool_model)